In order for this script to work:
Data must be exported in column format (how the data is displayed on the actual software)
All variables must be exported

In [1]:
# Import necessary packages 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/Users/ellacho/Downloads/10252023_treatment_UCN3_100ug_CHOW.csv', sep=',')

In [3]:
boxes = np.array(["Box 1", "Box 2", "Box 3", "Box 4", "Box 5", "Box 6", "Box 7", "Box 8"])

In [4]:
# Separate data by box

boxdata = []
for i in range(len(boxes)):
    boxdata.append(data.iloc[:,data.columns.get_loc(boxes[i]):data.columns.get_loc(boxes[i])+34])
    
# boxdata[#] gives you data per box

In [5]:
for i in range(len(boxdata)):
    
    # Get rid of irrelevant rows
    new_header = boxdata[i].iloc[0] #grab the first row for the header
    boxdata[i] = boxdata[i][1:] #take the data less the header row
    boxdata[i].columns = new_header #set the header row as the df header
    boxdata[i] = boxdata[i].iloc[1: , :] # drop unit row
    
    # Add date and time columns to each box's data
    boxdata[i]['Date'] = data['Date']
    boxdata[i]['Time'] = data['Time']
    
    # Change relevant variables to numeric
    boxdata[i][['Feed1']]= boxdata[i][['Feed1']].apply(pd.to_numeric)
    boxdata[i][['Drink1']]= boxdata[i][['Drink1']].apply(pd.to_numeric)
    boxdata[i][['DistK']]= boxdata[i][['DistK']].apply(pd.to_numeric)
    boxdata[i][['RER']]= boxdata[i][['RER']].apply(pd.to_numeric)
    boxdata[i][['VO2(1)']]= boxdata[i][['VO2(1)']].apply(pd.to_numeric)
    boxdata[i][['VCO2(1)']]= boxdata[i][['VCO2(1)']].apply(pd.to_numeric)
    boxdata[i][['Time']]= boxdata[i][['Time']].apply(pd. to_datetime)
    boxdata[i][['Date']]= boxdata[i][['Date']].apply(pd. to_datetime)
    
    # Evaluate intake and distance in each 9 min interval
#     boxdata[i]['Food Intake per Int'] = boxdata[i]['Feed1'].diff()
#     boxdata[i]['Water Intake per Int'] = boxdata[i]['Drink1'].diff()
    boxdata[i]['DistK'] = boxdata[i]['DistK'].diff()

In [6]:
# Pull out relevant variables only
boxdata_relevant = []

for i in range(len(boxdata)):
    boxdata_relevant.append(boxdata[i][['Date', 
                                        'Time', 
                                        'Feed1', 
                                        'Drink1', 
                                        'DistK', 
                                        'RER',
                                       'VO2(1)',
                                       'VCO2(1)']])

In [7]:
# Define a function to evaluate data per date; adjust evaluations as necessary

def date_data(boxdata_array, date_array):
    total_foodintake = np.zeros(len(date_array))
    total_drinkintake = np.zeros(len(date_array))
    total_DistK = np.zeros(len(date_array))
    total_VO2 = np.zeros(len(date_array))
    total_VCO2 = np.zeros(len(date_array))
    total_RER = np.zeros(len(date_array))

    for i in range(len(date_array)):
        total_foodintake[i] = boxdata_array['Feed1'].iloc[date_array[i]]
        total_drinkintake[i] = boxdata_array['Drink1'].iloc[date_array[i]]
        total_DistK[i] = boxdata_array['DistK'].iloc[date_array[i:i+1]].sum()
        total_VO2[i] = boxdata_array['VO2(1)'].iloc[date_array[i:i+1]].mean()
        total_VCO2[i] = boxdata_array['VCO2(1)'].iloc[date_array[i:i+1]].mean()
        total_RER[i] = boxdata_array['RER'].iloc[date_array[i:i+1]].mean()

    return date_array, total_foodintake, total_drinkintake, total_DistK, total_VO2, total_VCO2, total_RER

In [14]:
# Create array of days and params

indices_12hour = np.arange(0, 1120, 80) # array every 12 hrs 
params = np.array(['Cum. Food Intake', 'Cum. Drink Intake', 'DistK per 12hr', 
                   'Avg VO2', 'Avg VCO2', 'Avg RER'])

In [15]:
# Apply date data function to evaluate all of those parameters for each box

boxdata_relevant_day = []

for i in range(len(boxdata_relevant)):
    boxdata_relevant_day.append(date_data(boxdata_relevant[i], indices_12hour))
    
# Output: boxdata_relevant_day[x]; x = box number. 
# Outputs all arrays of relevant variables with measurements per date

In [16]:
# Create function to change data into dataframes

def transform_to_df(boxdata_date_array):
    boxdata_date_array = pd.DataFrame(data=boxdata_date_array)
    new_new_header = boxdata_date_array.iloc[0]
    boxdata_date_array = boxdata_date_array[1:] 
    boxdata_date_array.columns = new_new_header
    boxdata_date_array.insert(0, 'Params', params)
    
    return boxdata_date_array

In [17]:
# Change data into dataframes 

finaldata = []

for i in range(len(boxdata_relevant_day)):
    finaldata.append(transform_to_df(boxdata_relevant_day[i]))

In [18]:
# Create box file names
boxes_files = np.array(["Box1.xlsx", "Box2.xlsx", "Box3.xlsx", "Box4.xlsx", 
                        "Box5.xlsx", "Box6.xlsx", "Box7.xlsx", "Box8.xlsx"])

boxes_finaldata = np.array(["Box1data_int.xlsx", "Box2data_int.xlsx", "Box3data_int.xlsx", "Box4data_int.xlsx", 
                        "Box5data_int.xlsx", "Box6data_int.xlsx", "Box7data_int.xlsx", "Box8data_int.xlsx"])

In [19]:
# Export all individual box data
for i in range(len(boxdata_relevant)):
    boxdata_relevant[i].to_excel(boxes_files[i])
    finaldata[i].to_excel(boxes_finaldata[i])